In [1]:
import numpy as np
import pandas as pd

In [2]:
catalog = "../../Bases_de_datos/Data_catalog.csv"
iso = "../../Bases_de_datos/Country_ISO_code.csv"
path_v = "../data/variables.csv"
ix = "country_id"

df_c = pd.read_csv(catalog)
df_iso = pd.read_csv(iso)
data_s = pd.DataFrame(index = df_iso["alpha-3"])
data_s.index = data_s.index.set_names(ix)
data_c = data_s.copy()
data_p = data_s.copy()
data_q_s = data_s.copy()
data_q_c = data_s.copy()
data_q_p = data_s.copy()

df_v = pd.read_csv(path_v)
df_v = df_v[ df_v["Category"] == "population" ]

df_c[ df_c["Origin"] == "UNESCO UIS" ]


,Dataset,ID,Origin,Countries,Granularity,Timestep,Annex,Website,Filename,Path
20,Demographic and Socio-economic Indicators,DEM,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,DEM_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/DEM/
21,Other Policy Relevant Indicators (OPRI),OPRI,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,OPRI_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/OPRI/
22,Research and Development (R&D) – Other Policy ...,SCN-OPRI,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SCN-OPRI_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SCN-OPRI/
23,Research and Development (R&D) SDG 9.5,SCN-SDG,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SCN-SDG_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SCN-SDG/
24,SDG Global and Thematic Indicators,SDG,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SDG_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SDG/
25,SDG 11.4,SDG11,UNESCO UIS,All,National,Yearly,NaN,https://uis.unesco.org/bdds,SDG11_DATA_NATIONAL.csv,Bases_de_datos/UNESCO_UIS/SDG11/


In [69]:
for row in df_v[ df_v["ID"].isin(["SI.POV.GINI", "DEM", "SCN-SDG", "SDG"]) ].itertuples():
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0], skiprows = nr )
    
    if row.ID in ["DEM", "SCN-SDG"]:
        df.columns = [ x.lower() for x in df.columns ]

    if row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix}).iloc[:, :-1]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_s[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]
    elif row.Origin == "UNESCO UIS":
        if row.PERCENTAGE == "Yes": pop = data_s["200101"]
        else: pop = 1
        data_s[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_s[v] = ( 1 + pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_s[v] = 21 - data_q_s[v]

data_q_s

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,...,EA.6T8.AG25T99.F,EA.6T8.AG25T99.Q1,EA.6T8.AG25T99.RUR,ROFST.H.1,ROFST.H.1.F,ROFST.H.1.Q1,ROFST.H.1.RUR,XGDP.FSGOV,EXPGDP.TOT,Gini index
country_id,,,,,,,,,,,,,,,,,,,,,
AFG,4.0,20.0,20.0,20.0,1.0,20.0,NaN,18.0,2.0,3.0,...,3.0,10.0,6.0,1.0,1.0,1.0,1.0,11.0,NaN,NaN
ALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,12.0,8.0,5.0,8.0,10.0,16.0,14.0,6.0,18.0,20.0,...,13.0,16.0,11.0,12.0,12.0,12.0,10.0,5.0,5.0,17.0
DZA,3.0,9.0,12.0,20.0,8.0,17.0,13.0,6.0,6.0,6.0,...,10.0,16.0,12.0,14.0,14.0,13.0,14.0,18.0,12.0,19.0
ASM,19.0,11.0,12.0,NaN,13.0,NaN,NaN,NaN,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLF,20.0,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ESH,15.0,10.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,4.0,19.0,19.0,19.0,2.0,18.0,4.0,17.0,4.0,4.0,...,3.0,NaN,7.0,3.0,3.0,3.0,4.0,16.0,NaN,10.0


In [4]:
for row in df_v[ df_v["ID"].isin(["SI.POV.GINI", "DEM", "SCN-SDG", "SDG", "DF_GREEN_GROWTH"]) ].itertuples():
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0], skiprows = nr )
    
    if row.ID in ["DEM", "SCN-SDG"]:
        df.columns = [ x.lower() for x in df.columns ]

    if row.ID == "DF_GREEN_GROWTH":
        df = df[ df["MEASURE"] == v ]
        df = df.rename(columns = {"REF_AREA": ix}).iloc[:, :-1]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_s[v] = np.abs( df[ df["MEASURE"]==v
            ].sort_values( [ix, "TIME_PERIOD"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"] )
    elif row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix}).iloc[:, :-1]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_s[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]
    elif row.Origin == "UNESCO UIS":
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        if row.PERCENTAGE == "Yes": pop = data_s["200101"]
        else: pop = 1
        data_s[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_s[v] = ( 1 + pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_s[v] = 21 - data_q_s[v]

data_q_s["Mean"] = data_q_s.mean(axis = 1)
data_q_s["Count"] = data_q_s.count(axis = 1)
data_q_s = data_q_s[ data_q_s["Count"] > 1 ].sort_values("Mean")

data_q_s

/var/folders/td/smccb02154l21rhgckcmbs940000gn/T/ipykernel_14553/3520135678.py:6: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv( "../../"


,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,...,ROFST.H.1.F,ROFST.H.1.Q1,ROFST.H.1.RUR,XGDP.FSGOV,POP_NETMIGR,Subnational HDI,Subnational GDI,Gini index,Mean,Count
country_id,,,,,,,,,,,,,,,,,,,,,
VAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.000000,2
SSD,8.0,20.0,18.0,NaN,3.0,3.0,1.0,20.0,2.0,6.0,...,1.0,1.0,1.0,1.0,5.0,1.0,1.0,4.0,5.090909,23
AGO,4.0,16.0,19.0,2.0,6.0,4.0,5.0,18.0,1.0,1.0,...,4.0,4.0,2.0,3.0,5.0,5.0,5.0,1.0,5.166667,25
TCD,6.0,17.0,20.0,17.0,2.0,5.0,3.0,20.0,1.0,1.0,...,1.0,1.0,1.0,4.0,10.0,1.0,1.0,9.0,5.400000,26
PNG,8.0,17.0,17.0,2.0,6.0,6.0,2.0,16.0,6.0,5.0,...,2.0,3.0,4.0,1.0,9.0,4.0,5.0,5.0,5.520000,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SXM,19.0,20.0,8.0,NaN,17.0,NaN,NaN,8.0,15.0,8.0,...,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,14.777778,10
PLW,19.0,9.0,9.0,NaN,13.0,NaN,NaN,14.0,20.0,16.0,...,NaN,NaN,NaN,19.0,20.0,NaN,NaN,NaN,14.833333,13
ISL,16.0,7.0,6.0,NaN,19.0,18.0,20.0,2.0,12.0,3.0,...,NaN,NaN,NaN,19.0,17.0,20.0,12.0,20.0,15.105263,20


In [3]:
for row in df_v[ df_v["ID"].isin(["SI.POV.GINI", "DEM", "SCN-SDG", "SDG", "DF_GREEN_GROWTH", "Subnational_HDI"]) ].itertuples():
    v = row.Variable
    if row.Origin == "World Bank": nr = 3
    else: nr = 0
    
    df = pd.read_csv( "../../"
        + df_c.loc[df_c["ID"]==row.ID, 'Path'].iloc[0]
        + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0], skiprows = nr )
    
    if row.Origin == "OECD":
        if row.ID == "DF_GREEN_GROWTH":
            df = df[ df["MEASURE"] == v ]
            df = df.rename(columns = {"REF_AREA": ix}).iloc[:, :-1]
            df = df[ df[ix].isin(df_iso["alpha-3"]) ]
            data_s[v] = np.abs( df[ df["MEASURE"]==v
                ].sort_values( [ix, "TIME_PERIOD"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"] )
    if row.Origin == "Global Data Lab":
        df = df.rename(columns = {"ISO_Code": ix}).iloc[:, :-1]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_s[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
    elif row.Origin == "World Bank":
        df = df.rename(columns = {"Country Code": ix}).iloc[:, :-1]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        data_s[v] = df.set_index(ix).iloc[:, 4:-1].ffill(axis = 1).iloc[:, -1]
    elif row.Origin == "UNESCO UIS":
        if row.ID in ["DEM", "SCN-SDG"]:
            df.columns = [ x.lower() for x in df.columns ]
        df = df[ df[ix].isin(df_iso["alpha-3"]) ]
        if row.PERCENTAGE == "Yes": pop = data_s["200101"]
        else: pop = 1
        data_s[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
            ).drop_duplicates(ix, keep="last").set_index(ix)["value"] / pop )
    
    # corregir random
    data_q_s[v] = ( 1 + pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False ) )
    if row.Negative_indicator:
        data_q_s[v] = 21 - data_q_s[v]

data_q_s

/var/folders/td/smccb02154l21rhgckcmbs940000gn/T/ipykernel_14553/2264041609.py:6: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv( "../../"


,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,...,EA.6T8.AG25T99.RUR,ROFST.H.1,ROFST.H.1.F,ROFST.H.1.Q1,ROFST.H.1.RUR,XGDP.FSGOV,POP_NETMIGR,Subnational HDI,Subnational GDI,Gini index
country_id,,,,,,,,,,,,,,,,,,,,,
AFG,4.0,20.0,20.0,20.0,1.0,20.0,NaN,18.0,2.0,3.0,...,6.0,1.0,1.0,1.0,1.0,11.0,2.0,2.0,1.0,NaN
ALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,12.0,8.0,5.0,8.0,10.0,20.0,14.0,6.0,18.0,20.0,...,11.0,12.0,12.0,12.0,10.0,5.0,9.0,14.0,19.0,17.0
DZA,3.0,9.0,12.0,20.0,8.0,19.0,13.0,6.0,6.0,6.0,...,12.0,14.0,14.0,13.0,14.0,18.0,7.0,11.0,3.0,19.0
ASM,19.0,11.0,12.0,NaN,13.0,NaN,NaN,NaN,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,20.0,14.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLF,20.0,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN
ESH,15.0,10.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,4.0,19.0,19.0,19.0,2.0,18.0,4.0,17.0,4.0,4.0,...,7.0,3.0,3.0,3.0,4.0,16.0,2.0,1.0,1.0,10.0


In [107]:
# UNESCO UIS
# DEM: Demographic and Socio-economic Indicators
n = 20
file = f"../../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename" ]}"
df = pd.read_csv(file)
df.columns = [ x.lower() for x in df.columns ]

df_v[ df_v["ID"] == "DEM" ]

vars = [
    "200101",
    "200144",
    "200151",
    "DT.TDS.DECT.GN.ZS",
    "NY.GDP.PCAP.CD",
    "SH.DYN.AIDS.ZS",
    "SI.POV.LMIC",
    "SP.DYN.LE00.IN",
    "SP.DYN.TFRT.IN",
    "SP.POP.GROW",
    "SP.RUR.TOTL.ZS",
    ]
names = [
    "Total population (thousands)",
    "Population aged 15-24 years (thousands)",
    "Population aged 65 years or older (thousands)",
    "Total debt service (% of GNI)",
    "GDP per capita (current US$)",
    "Prevalence of HIV, total (% of population ages 15-49)", # SDG 3.3.1
    "Poverty headcount ratio at $3.20 a day (PPP) (% of population)", # similar a SDG 1.1.1
    "Mortality rate, infant (per 1,000 live births)", # SDG 3.2.1
    "Life expectancy at birth, total (years)",
    "Fertility rate, total (births per woman)",
    "Population growth (annual %)",
    "Rural population (% of total population)"
    ]
invert = [ 0, 2, 3, 5, 6, 7, 8, 9, 10 ]

for i, v in enumerate(vars):
    data_s[v] = df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
        ).drop_duplicates(ix, keep="last").set_index(ix)["value"]
    #if v in vars[1:3]:
    #    data_s[v] = data_s[v] / data_s[vars[0]]
    data_q_s[v] = pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False )
    if i in invert:
        data_q_s[v] = 19 - data_q_s[v]

data_q_s

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,SP.RUR.TOTL.ZS
country_id,,,,,,,,,,,
AFG,3.0,17.0,6.0,19.0,0.0,18.0,NaN,17.0,1.0,2.0,1.0
ALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,11.0,7.0,9.0,7.0,9.0,17.0,13.0,5.0,17.0,19.0,10.0
DZA,2.0,16.0,3.0,19.0,7.0,16.0,12.0,5.0,5.0,5.0,13.0
ASM,18.0,2.0,18.0,NaN,12.0,NaN,NaN,NaN,NaN,19.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
WLF,19.0,0.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ESH,14.0,5.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,3.0,17.0,6.0,18.0,1.0,15.0,3.0,16.0,3.0,3.0,4.0


In [113]:
# UNESCO UIS
# SCN-OPRI: Research and Development (R&D) – Other Policy Relevant Indicators
n = 22
file = f"../../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename"]}"
df = pd.read_csv(file)
df.columns = [ x.lower() for x in df.columns ]

vars = [
    "FRESP.TFTE"
    ]
names = [
    "Researchers (FTE) - % Female", # FTE: Full Time Equivalent
    ]

for v in vars:
    data_s[v] = df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
        ).drop_duplicates(ix, keep="last").set_index(ix)["value"]
    data_q_s[v] = pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False )

data_q_s

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,SP.RUR.TOTL.ZS,FRESP.TFTE
country_id,,,,,,,,,,,,
AFG,3.0,19.0,19.0,19.0,0.0,16.0,NaN,17.0,1.0,2.0,1.0,NaN
ALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,11.0,7.0,4.0,7.0,9.0,17.0,13.0,5.0,17.0,19.0,10.0,13.0
DZA,2.0,8.0,11.0,19.0,7.0,16.0,12.0,5.0,5.0,5.0,13.0,15.0
ASM,18.0,10.0,11.0,NaN,12.0,NaN,NaN,NaN,NaN,19.0,16.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
WLF,19.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ESH,14.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,3.0,18.0,18.0,18.0,1.0,19.0,3.0,16.0,3.0,3.0,4.0,NaN


In [143]:
# UNESCO UIS
# SCN-SDG: Research and Development (R&D) SDG 9.5
n = 23
file = f"../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename"]}"
df = pd.read_csv(file)
df.columns = [ x.lower() for x in df.columns ]

vars = [
    "EXPGDP.TOT",
    "RESDEN.INHAB.TFTE"
    ]
names = [
    "GERD as a percentage of GDP", # SDG 9.5.1
    "Researchers per million inhabitants (FTE)", # SDG 9.5.2
    ]

for v in vars:
    data_s[v] = df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
        ).drop_duplicates(ix, keep="last").set_index(ix)["value"]
    data_q_s[v] = pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False )

In [9]:
# UNESCO UIS
# SDG: SDG Global and Thematic Indicators
# SDG 4: Education

n = 24
file = f"../../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename"]}"
df = pd.read_csv(file)
df.columns = [ x.lower() for x in df.columns ]

vars = [
    "EA.6T8.AG25T99",
    "EA.6T8.AG25T99.F",
    "EA.6T8.AG25T99.Q1",
    "EA.6T8.AG25T99.RUR",
    "XGDP.FSGOV"
    ]
names = [
    # SDG 4.4.3
    "Educational attainment rate, completed Bachelor's or equivalent education or higher, population 25+ years, both sexes (%)",
    "Educational attainment rate, completed Bachelor's or equivalent education or higher, population 25+ years, female (%)",
    "Educational attainment rate, completed Bachelor's or equivalent education or higher, population 25+ years, poorest quintile, both sexes (%)",
    "Educational attainment rate, completed Bachelor's or equivalent education or higher, population 25+ years, rural, both sexes (%)",
    # SDG 4.F.1
    "Government expenditure on education as a percentage of GDP (%)",
    ]

for v in vars:
    data_s[v] = df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
        ).drop_duplicates(ix, keep="last").set_index(ix)["value"]
    data_q_s[v] = pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False )
    
data_q_s

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,SP.RUR.TOTL.ZS,EA.6T8.AG25T99,EA.6T8.AG25T99.F,EA.6T8.AG25T99.Q1,EA.6T8.AG25T99.RUR,ROFST.H.1,ROFST.H.1.F,ROFST.H.1.Q1,ROFST.H.1.RUR,XGDP.FSGOV
country_id,,,,,,,,,,,,,,,,,,,,
AFG,3.0,19.0,19.0,19.0,0.0,19.0,NaN,17.0,1.0,2.0,1.0,3.0,2.0,9.0,5.0,0.0,0.0,0.0,0.0,10.0
ALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,11.0,7.0,4.0,7.0,9.0,18.0,13.0,5.0,17.0,19.0,10.0,11.0,12.0,15.0,10.0,11.0,11.0,11.0,9.0,4.0
DZA,2.0,8.0,11.0,19.0,7.0,16.0,12.0,5.0,5.0,5.0,13.0,8.0,9.0,15.0,11.0,13.0,13.0,12.0,13.0,17.0
ASM,18.0,10.0,11.0,NaN,12.0,NaN,NaN,NaN,NaN,19.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLF,19.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ESH,14.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,3.0,18.0,18.0,18.0,1.0,16.0,3.0,16.0,3.0,3.0,4.0,4.0,2.0,NaN,6.0,2.0,2.0,2.0,3.0,15.0


In [145]:
# UNESCO UIS
# SDG 11.4
# Cultural and natural heritage SDG target

n = 25
file = f"../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename"]}"
df = pd.read_csv(file)
df.columns = [ x.lower() for x in df.columns ]

vars = [
    "HEXPCSTPPPCAP.CULHER",
    "HEXPCSTPPPCAP.NATHER"
    ]
names = [
    # SDG 11.4.1
    "Total expenditure  (public and private) per capita spent on cultural heritage (constant PPP$ - 2017)",
    "Total expenditure (public and private) per capita spent on natural heritage (constant PPP$ - 2017)",
    ]

for v in vars:
    data_s[v] = df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
        ).drop_duplicates(ix, keep="last").set_index(ix)["value"]
    data_q_s[v] = pd.qcut( data_s[v]
        + np.random.random(data_s[v].shape[0])/10000, 20, labels = False )

In [146]:
data_q_s["Mean"] = data_q_s.mean(axis = 1)
data_q_s["Count"] = data_q_s.count(axis = 1)
data_q_s = data_q_s[ data_q_s["Count"] > 1 ].sort_values("Mean")

data_q_s

,200101,200144,200151,DT.TDS.DECT.GN.ZS,NY.GDP.PCAP.CD,SH.DYN.AIDS.ZS,SI.POV.LMIC,SP.DYN.LE00.IN,SP.DYN.TFRT.IN,SP.POP.GROW,...,RESDEN.INHAB.TFTE,EA.6T8.AG25T99,EA.6T8.AG25T99.F,EA.6T8.AG25T99.Q1,EA.6T8.AG25T99.RUR,XGDP.FSGOV,HEXPCSTPPPCAP.CULHER,HEXPCSTPPPCAP.NATHER,Mean,Count
country_id,,,,,,,,,,,,,,,,,,,,,
VAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.000000,2
AGO,3.0,15.0,18.0,1.0,5.0,3.0,4.0,17.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,2.0,NaN,NaN,4.611111,19
TZA,1.0,17.0,17.0,11.0,2.0,1.0,1.0,14.0,1.0,0.0,...,0.0,1.0,1.0,2.0,2.0,5.0,NaN,NaN,4.894737,20
PNG,7.0,16.0,16.0,1.0,5.0,6.0,1.0,15.0,5.0,4.0,...,1.0,2.0,2.0,3.0,4.0,0.0,NaN,NaN,4.894737,20
SHN,19.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.200000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHE,8.0,2.0,3.0,NaN,19.0,14.0,19.0,0.0,16.0,11.0,...,18.0,19.0,17.0,NaN,19.0,15.0,14.0,18.0,13.500000,19
CUW,16.0,8.0,5.0,NaN,13.0,NaN,NaN,4.0,17.0,19.0,...,NaN,16.0,16.0,NaN,NaN,18.0,NaN,NaN,13.545455,12
GUF,15.0,14.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.666667,4


In [147]:
df_c[ df_c["Origin"] == "OECD" ]

,Dataset,ID,Origin,Countries,Granularity,Timestep,Annex,Website,Filename,Path
0,Air Emissions Accounts,DF_AEA,OECD,All,National,"OECD: Quarterly, Non-OECD: Yearly",NaN,https://data-explorer.oecd.org,"OECD.SDD.NAD.SEEA,DSD_AEA@DF_AEA,1.0+all.csv",Bases_de_datos/OECD_Air_and_Climate/
1,Air emissions - Air pollutants Inventories,DF_AIR_EMISSIONS,OECD,All,National,Yearly,Air Emissions Correspondance Table.xlsx,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_AIR_EMISSIONS@DF_AIR_EMISSION...",Bases_de_datos/OECD_Air_and_Climate/
2,Air emissions - Greenhouse gas emissions Inven...,DF_AIR_GHG,OECD,All,National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_AIR_GHG@DF_AIR_GHG,1.0+all.csv",Bases_de_datos/OECD_Air_and_Climate/
3,Air transport CO2 emissions (experimental),DF_AIR_TRANSPORT,OECD,All,National,Monthly,NaN,https://data-explorer.oecd.org,"OECD.SDD.NAD.SEEA,DSD_AIR_TRANSPORT@DF_AIR_TRA...",Bases_de_datos/OECD_Air_and_Climate/
4,Maritime transport CO2 emissions (experimental),DF_MARITIME_TRANSPORT,OECD,All,National,Monthly,NaN,https://data-explorer.oecd.org,"OECD.SDD.NAD.SEEA,DSD_MARITIME_TRANSPORT@DF_MA...",Bases_de_datos/OECD_Air_and_Climate/
5,Exposure to air pollution,DF_AIR_POLL,OECD,"OECD, Argentina, Brazil, China, India, Indones...",Subnational; National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_AIR_POL@DF_AIR_POLL,1.0+all.csv",Bases_de_datos/OECD_Air_and_Climate/
6,Exposure to extreme temperature,EXT_TEMP,OECD,"OECD, Argentina, Brazil, China, India, Indones...",Subnational; National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_ECH@EXT_TEMP,1.1+all.csv",Bases_de_datos/OECD_Air_and_Climate/
7,Exposure to coastal flooding,COAS_FLOOD,OECD,"OECD, Argentina, Brazil, China, India, Indones...",Subnational; National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_ECH@COAS_FLOOD,1.1+all.csv",Bases_de_datos/OECD_Air_and_Climate/
8,Exposure to drought,EXT_DROUGHT,OECD,"OECD, Argentina, Brazil, China, India, Indones...",Subnational; National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_ECH@EXT_DROUGHT,1.1+all.csv",Bases_de_datos/OECD_Air_and_Climate/
9,Exposure to extreme precipitation,EXT_PREC,OECD,"OECD, Argentina, Brazil, China, India, Indones...",Subnational; National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_ECH@EXT_PREC,1.1+all.csv",Bases_de_datos/OECD_Air_and_Climate/


In [179]:
n = 5
file = f"../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename"]}"
df = pd.read_csv(file)
print( df["MEASURE"].unique() )
print( df["Measure"].unique() )
print( df["POLLUTANT"].unique() )
print( df["Pollutant"].unique() )

['POP_EXP_POL' 'MEAN_POP']
['Population share exposed' 'Mean population-weighted exposure']
['PM2_5']
['Fine particulate matter (PM2.5)']


In [149]:
# OECD
# EXT_TEMP: Exposure to extreme temperature

n = 6
file = f"../{df_c.loc[n, "Path"]}{df_c.loc[n, "Filename"]}"
df = pd.read_csv(file)
df = df.rename( columns = {"REF_AREA": ix} )
df = df[ df[ix].isin(df_iso["alpha-3"]) ]

vars = [
    "TN_POP_EXP",
    "HD_TN_POP_EXP",
    "HD_POP_EXP",
    "ID_POP_EXP",
    "UTCI_POP_EXP",
    "TEMP_CHANGE"
    ]
names = [
    "Population exposure to tropical nights",
    "Population exposure to days identified to be a hot summer day and tropical night",
    "Population exposure to hot days",
    "Population exposure to icing days",
    "Mean population exposure to heat stress",
    "Annual temperature change",
    ]

for v in vars:
    data_c[v] = df[ df["MEASURE"]==v ].sort_values( [ix, "TIME_PERIOD"]
        ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]
    data_q_c[v] = pd.qcut( data_c[v]
            + np.random.random(data_c[v].shape[0])/10000, 20, labels = False )

In [150]:
n = 6
file = f"../{df_c.loc[n, "Path"]}{df_c.loc[ n, "Filename" ]}"
df = pd.read_csv(file)
df[ df["REF_AREA"].isin(df_iso["alpha-3"]) ]

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,FREQ,Frequency of observation,MEASURE,Measure,...,TIME_PERIOD,Time period,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals
386,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1979,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
387,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1980,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
388,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1981,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
389,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1982,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
390,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,SXM,Sint Maarten,A,Annual,ID_POP_EXP,Population exposure to icing days,...,1983,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125645,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2018,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
1125646,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2019,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
1125647,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2020,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
1125648,DATAFLOW,OECD.ENV.EPI:DSD_ECH@EXT_TEMP(1.1),Exposure to extreme temperature,I,FLK,Falkland Islands (Malvinas),A,Annual,TN_POP_EXP,Population exposure to tropical nights,...,2021,NaN,0.0,NaN,NaN,NaN,0,Units,2,Two
